In [7]:
# Frog Game
import queue
import time
from z3 import *

class Node:

    def __init__(self, locData, original, lightData, leadData):
        self.depth = 0
        self.locData = locData
        self.children = []
        self.parent = None
        self.original = original
        self.lightLoc = lightData
        self.leadLoc = leadData
            
    def bfPrintTree(self):
        
        # creates queue for BFTraversal
        nodeHolder = queue.Queue(0)
        
        # puts root onto the queue
        nodeHolder.put(self)
        prevChildNum = 1
        
        while(nodeHolder.empty() == False):
            
            currNode = nodeHolder.get() # we need a copy of this to work with
            
            # loops through all its children and adds them to queue
            numChildren = 0
            for c in  currNode.children:
                nodeHolder.put(c)
                numChildren = numChildren + 1
            #if(currNode.left != None):
            #    nodeHolder.put(currNode.left)
            #if(currNode.right != None):
            #    nodeHolder.put(currNode.right)
            
                
            print("(", currNode.depth, "): ", currNode.locData) # then here we can finally remove with .get()
                
            prevChildNum = numChildren
            
    def printUpPath(self):
        
        if(self.parent == None):
            # means we got to the light
            print(self.locData)
            
            size = len(self.locData)
            originalLoc = []
            print(self.original)  
            print("EOP")
            return
        else:
            # means we're either at 1st node or something in the middle
            print(self.locData)
            self.parent.printUpPath()
            
        
    def generateTree(self, padFill, numPads, lightFill, leadFill):
        # padFill represents the stacks of frogs, if padFill[i] = 0 no frogs
        # for each stack of frogs, we need to generate the possible move trees
        
        padNum = 0 # to keep track of which pad the stack is on
        
        for s in padFill:
            # no frogs in the stack do nothing
            if(s != 0):
                # found a pad with frogs on it, now generate the moves
                # HERE IS WHERE ALL THE MOVEMENT RULES NEED TO GO
                
                # calculate a possible left move and a possible right move
                # then ensure that move within bounds / goes to a pad with frogs on it
                    # current stack location is padNum
                    # current amt of frogs is s
                leftLoc = padNum - s # current location - number of frogs
                rightLoc = padNum + s # current location + number of frogs
                
                # if the proposed pad exists w/in bounds and has frog(s) on it
                # and also the proposed pad is not a light frog
                # and also our current pad is not a lead frog
                if(leftLoc >= 0 and padFill[leftLoc] > 0 and leadFill[padNum] != 1): 
                    
                    if(lightFill[leftLoc] != 1):
                        padFillL = padFill.copy()
                        lightFillL = lightFill.copy()
                        padFillL[padNum] = 0 # after succesful move, current pad will empty
                        padFillL[leftLoc] = padFill[leftLoc] + s # and proposed left location will have its stack + moved stack

                        # if current pad has a light frog on it, need to update lightFill
                        if(lightFill[padNum] == 1):
                            lightFillL[padNum] = 0 # now original spot has no light frog
                            lightFillL[leftLoc] = 1 # and it has moved to the left location

                        # now create the new node representing this left movement path
                        newNodeL = Node(padFillL, padFill, lightFillL, leadFill)
                        newNodeL.depth = self.depth + 1
                        newNodeL.parent = self
                        self.children.append(newNodeL)
                        # and attach it to the root accordingly
                        newNodeL.generateTree(padFillL, numPads, lightFillL, leadFill)
                    
                    
                # if the proposed pad exists w/in bounds and has frog(s) on it
                # and current pad is not a lead frog
                # and rightLoc pad does not have a light frog
                if(rightLoc <= numPads-1 and padFill[rightLoc] > 0 and leadFill[padNum] != 1): 
                    
                    if(lightFill[rightLoc] != 1):
                        padFillR = padFill.copy()
                        leadFillR = leadFill.copy()
                        padFillR[padNum] = 0 # after succesful move, current pad will empty
                        padFillR[rightLoc] = padFill[rightLoc] + s # and proposed right location will have its stack + moved stack

                        # if current pad has a light frog on it, need to update lightFill
                        if(leadFill[padNum] == 1):
                            leadFillR[padNum] = 0 # now original spot has no light frog
                            leadFillR[rightLoc] = 1 # and it has moved to the left location

                        # now create the new node representing this left movement path
                        newNodeR = Node(padFillR, padFill, lightFill, leadFillR)
                        newNodeR.depth = self.depth + 1
                        newNodeR.parent = self
                        self.children.append(newNodeR)
                        # and attach it to the root accordingly
                        newNodeR.generateTree(padFillR, numPads, lightFill, leadFillR)
                    
                # if able to move left generate tree as leftchild of root
                    # pad s to the left has frogs in it
                # if able to move right generate tree as rightchild of root
                    # pad s to the right has frogs in it
                # if unable to move, generate NONE as left and right children
                # do nothing
                
            # happens even if no frogs on pad
            padNum = padNum + 1
        
        # outside the for loop
    
    def getLeafNodes(self):
        
        leafList = []
        
        
        selfHadChild = False
        
        # for each child, check if it has any children, if not its a leaf, if so, get the leaf nodes of it recursivly
        for c in self.children: # checks self's list of children
            selfHadChild = True
            numNull = 0
            numChild = 0
            for c_2 in c.children: # checks child's list of children
                # if at anypoint it finds a child who is not None, we get the leaf nodes of current child of self
                if(c_2 == None):
                    numNull = numNull + 1
                else:
                    # if c's children existed
                    numChild = numChild + 1
                
            # after searching current child of self's list of children, either found it to be a leaf or a parent
            if(numChild != 0): # means Node was a parent
                # so we need to get its leaf nodes too
                leafList.extend(c.getLeafNodes())
            else: # means Node had no children
                # means c is a leaf so add it to list and return?
                leafList.append(c)
        
        # if self had no children, we need to add it alone to leaf list and appent
        if(~selfHadChild):
            leafList.append(self)
            
        return leafList
            
#########################
### END OF NODE CLASS ###
#########################

# Node.frog represents which frog moves during to create the next level of tree
# Node.locData represents the current amounts of frogs on each pad

def makeAllMoves(numPads, padFill, lightFill, leadFill, realNumFrogs):
    
    ### PRE CONDITIONS ###
    for i in padFill:
        if(i != 1 and i != 0):
            print("Error, invalid padFill")
            return None
    for i in lightFill:
        if(i != 1 and i != 0):
            print("Error, invalid lightFill")
            return None
    for i in leadFill:
        if(i != 1 and i != 0):
            print("Error, invalid leadFill")
            return None
    #######################
    
    pathList = [] # list to hold all possible paths of movement for the given frog arrangement
    # generate a tree for each possible 1st move
    # loop through each frog
    for f in range(0, numPads): # 0, 1, 2, 3, ... , numPads-1
        #loop through each possible move
        # first and last frog only have 1 move Right or Left
        if(f == 0):
            # now spot 0 has 0 frogs, spot 1 has 2
            # update location data and generate tree
            padFill0 = padFill.copy()
            lightFill0 = lightFill.copy()
            leadFill0 = leadFill.copy()

            # if pad[1] not waterlogged
            # need to ensure padFill[1] not a light frog
            # need to ensure padFill[0] not a lead frog
            # if what we move is a lightFrog (lead Frogs cannot move) need to update the lightFill list
            if(padFill[1] != 0 and padFill[0] != 0 and lightFill[1] != 1 and leadFill[0] != 1):
                padFill0[0] = 0
                padFill0[1] = 2

                # if frog we move is a light frog, need to update lightFill
                if(lightFill[0] == 1):
                    lightFill0[0] = 0  # now there is no light frog in the first spot
                    lightFill0[1] = 1  # now the light frog is in the spot to the right

                # now generate the tree using the current location data
                # create a root of the tree, in generateTree will make its children 
                root0 = Node(padFill0, padFill, lightFill0, leadFill0)
                root0.depth = 1
                root0.generateTree(padFill0, numPads, lightFill0, leadFill0)

                # add to list of possible path trees
                pathList.append(root0)

        elif(f == numPads-1): 

            # now spot numPadss-1 has 0 frogs, spot numPadss-2 has 2
            # update location data and generate tree
            padFillf = padFill.copy()
            lightFillf = lightFill.copy()
            leadFillf = leadFill.copy()

            # if pad[numPads-2] not waterlogged and pad[numPads-1]
            # need to ensure padFill[numPads-2] not a light frog
            # need to ensure padFill[numPads-1] not a lead frog
            if(padFill[numPads-2] != 0 and padFill[numPads-1] and lightFill[numPads-2] != 1 and leadFill[numPads-1] != 1):
                padFillf[numPads-1] = 0
                padFillf[numPads-2] = 2

                # if frog we move is a light frog, need to update lightFill
                if(lightFill[numPads-1] == 1):
                    lightFillf[numPads-1] = 0  # now there is no light frog in the last spot
                    lightFillf[numPads-2] = 1  # now the light frog is in the spot to the left
                # now generate the tree using the current location data
                # create a root of the tree, make left child the left move path, right child the right move path
                rootf = Node(padFillf, padFill, lightFillf, leadFillf)
                rootf.depth = 1
                rootf.generateTree(padFillf, numPads, lightFillf, leadFillf)



                # add to list of possible paths
                pathList.append(rootf)

        else:
            # need to generate two trees, one left one right
            # update location datas and generate trees
            padFillR = padFill.copy()
            padFillL = padFill.copy()
            lightFillR = lightFill.copy()
            leadFillR = leadFill.copy()
            lightFillL = lightFill.copy()
            leadFillL = leadFill.copy()

            # need seperate data for left and right because generate different trees of possible moves
            # able to move left
            if(padFill[f-1] != 0 and padFill[f] != 0 and lightFill[f-1] != 1 and leadFill[f] != 1):
                padFillL[f] = 0
                padFillL[f-1] = 2

                # if frog we move is a light frog, need to update lightFill
                if(lightFill[f] == 1):
                    lightFillL[f] = 0  # now there is no light frog in the first spot
                    lightFillL[f-1] = 1  # now the light frog is in the spot to the left

                # now generate the tree using the current location data
                # create a root of the tree, make left child the left move path, right child the right move path

                rootL = Node(padFillL, padFill, lightFill, leadFill)
                rootL.depth = 1
                rootL.generateTree(padFillL, numPads, lightFillL, leadFill)

                # add to list of possible paths

                pathList.append(rootL)

            if(padFill[f+1] != 0 and padFill[f] != 0 and lightFill[f+1] != 1 and leadFill[f] != 1):
                padFillR[f] = 0
                padFillR[f+1] = 2

                # if frog we move is a light frog, need to update lightFill
                if(lightFill[f] == 1):
                    lightFillR[f] = 0  # now there is no light frog in the first spot
                    lightFillR[f+1] = 1  # now the light frog is in the spot to the right

                rootR = Node(padFillR, padFill, lightFillR, leadFill)
                rootR.depth = 1
                rootR.generateTree(padFillR, numPads, lightFillR, leadFill)

                pathList.append(rootR)

    # succesfully calculates roots of all movement trees (root.locData = frog arrangement after 1st move)
    #print("Possible Paths: ")
    #count = 0
    leafList = []
    for p in pathList:
        leafList.extend(p.getLeafNodes())
        
    ### POST CONDITIONS ###
    for l in leafList:
        
        if(sum(l.locData) != realNumFrogs):
            print("Error, failed postcondition.")
            return None
        for i in l.lightLoc:
            if(i != 1 and i != 0):
                print("Error, failed postcondition. invalid lightFill")
                return None
        for i in l.leadLoc:
            if(i != 1 and i != 0):
                print("Error, failed postcondition. invalid leadFill")
                return None
    #######################

    # prints all possible paths from leaf nodes down to start
    #for l in leafList:
        #print(l.locData)
    #    l.printUpPath()

        #print("Path[", count, "]: ")
        #print("( 0 ): ", padFill)
        #p.bfPrintTree()
        #count = count + 1

    # need to find all leaf nodes and store them
    # because the leaf nodes are what we will add to the Z3 solver to output correct paths

    solList = []

    #print("Real num frogs: ", realNumFrogs)
    for l in leafList:
        #print("Leaf: ", l.locData)
        for v in l.locData:
            if(v == realNumFrogs):
                solList.append(l)
    
    return solList

#####################################
### END OF FUNCTION makeAllMoves()###
#####################################

def getUserInput():
    padFill = [] # location data of frogs, value in index represents number of frogs at pad_index
    lightFill = [] # location data of light frogs (frogs that cannot be jumped on)
    leadFill = [] # location data of lead frogs (frogs that cannot jump onto others)

    numPads = int(input("Enter number of pads: ")) # this really means the number of pads
    realNumFrogs = 0
    numlightFrogs = 0
    numleadFrogs = 0
    # for waterlogged pads, just need to start spot off with 0 frogs
    for f in range(0, numPads):
        print("Waterlog pad[", f, "]? No = 0, Yes = 1: ")
        pad = int(input("0/1?: "))
        if(pad == 0): # not a waterlog pad, so place a frog and a 1
            realNumFrogs = realNumFrogs + 1
            padFill.append(1)
        elif(pad == 1): # is a waterlog pad, so place no frog on the pad
            padFill.append(0)

        if(padFill[f] == 1):
            print("light Frog on pad[", f, "]? No = 0, Yes = 1")
            light = int(input("0/1?: "))
            if(light == 1): # if light frog on pad
                numlightFrogs = numlightFrogs + 1
                lightFill.append(1) # add 1 to lightFill list to represent its location
            elif(light == 0):
                lightFill.append(0) # if not on pad, add 0 to lightFill list to represent a light frogs nonexistance there

            if(lightFill[f] != 1):
                print("lead Frog on pad[", f, "]? No = 0, Yes = 1")
                lead = int(input("0/1?: "))
                if(lead == 1): # if light frog on pad
                    numleadFrogs = numleadFrogs + 1
                    leadFill.append(1) # add 1 to lightFill list to represent its location
                elif(lead == 0):
                    leadFill.append(0) # if not on pad, add 0 to lightFill list to represent a light frogs nonexistance there
            else:
                leadFill.append(0) # if already a light frog there, know we won't have a lead frog there
        else:
            lightFill.append(0)
            leadFill.append(0)
            # if it was a waterlog automatically we know there are not light or lead frogs in the spot
            
    print("\n")
    return [numPads, padFill, lightFill, leadFill, realNumFrogs]

#####################################
### END OF FUNCTION getUserInput()###
#####################################


################################
### START OF EXPERIMENTATION ###
################################

### CASE 0: User Input Arrangment
padNum, padFill, lightFill, leadFill, realNumFrogs = getUserInput()
solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)

# printing solution paths
for s in solList:
    s.printUpPath()

### CASE 1: regular frogs
"""print("Starting Case 1...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        padFill.append(1)
        realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print("Time to Generate: ", genTime, "   Num of Solutions: ", numSols)
    
print("\n")"""
 
### CASE 2: single waterlogged pad in middle
    # this case places it directly in the middle, if 3 pads puts it at 3//2 = 1.5 = 1 [0, 1, 2]
"""print("Starting Case 2...")
for padNum in range(3, 13):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == padNum // 2):
            padFill.append(0)
        else:
            padFill.append(1)
            realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print("Time to Generate: ", genTime, "   Num of Solutions: ", numSols)

print("\n")"""

### CASE 3: 2 adjacent waterlogged pads in the middle
"""print("Starting Case 3...")
for padNum in range(3, 13):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == (padNum // 2) - 1):
            padFill.append(0)
        elif(i == padNum // 2):
            padFill.append(0) # places waterlog pad next to center waterlog pad
        else:
            padFill.append(1)
            realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print("Time to Generate: ", genTime, "   Num of Solutions: ", numSols)

print("\n")"""

### CASE 4: 2 waterlogged pads in the middle with a single frog seperating them
"""print("Starting Case 4...")
for padNum in range(3, 13):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == (padNum // 2) - 1):
            padFill.append(0)
        elif(i == (padNum // 2) + 1):
            padFill.append(0) # places waterlog pad next to center waterlog pad with 1 frog between them
        else:
            padFill.append(1)
            realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print("Time to Generate: ", genTime, "   Num of Solutions: ", numSols)

print("\n")"""

### CASE 5: 3 adjacent waterlogged pads in the middle
"""print("Starting Case 5...")
for padNum in range(6, 15):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == (padNum // 2) - 1):
            padFill.append(0)
        elif(i == padNum // 2):
            padFill.append(0) # places waterlog pad next to center waterlog pad
        elif(i == (padNum // 2) + 1):
            padFill.append(0) # places waterlog pad next to other 2 waterlogged pads
        else:
            padFill.append(1)
            realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)

print("\n")"""

### CASE 6: 3 waterlogged pads in the middle with two frogs seperating them
"""print("Starting Case 6...")
for padNum in range(6, 15):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == (padNum // 2) - 2):
            padFill.append(0)
        elif(i == padNum // 2):
            padFill.append(0) # places waterlog pad next to center waterlog pad with frog between
        elif(i == (padNum // 2) + 2):
            padFill.append(0) # places waterlog pad next to other 2 waterlogged pads with frog between
        else:
            padFill.append(1)
            realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)

print("\n")"""

### CASE 7: light frog in 1st spot
"""print("Starting Case 7...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        padFill.append(1)
        realNumFrogs = realNumFrogs + 1
        leadFill.append(0)
        # places light frog only on 1st pad
        if(i == 0):
            lightFill.append(1)
        else:
            lightFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)
    
print("\n")"""

### CASE 8: light frog in middle spot
"""print("Starting Case 8...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == padNum // 2):
            lightFill.append(1)
        else:
            lightFill.append(0)    
        realNumFrogs = realNumFrogs + 1
        padFill.append(1)
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)

print("\n")"""

### CASE 9: lead frog in 1st spot
"""print("Starting Case 9...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        padFill.append(1)
        realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
        # places light frog only on 1st pad
        if(i == 0):
            leadFill.append(1)
        else:
            leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)
    
print("\n")"""

### CASE 10: lead frog in middle spot
"""print("Starting Case 10...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == padNum // 2):
            leadFill.append(1)
        else:
            leadFill.append(0)    
        realNumFrogs = realNumFrogs + 1
        padFill.append(1)
        lightFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)

print("\n")"""

### CASE 11: waterlogged pad in middle, light frog adjacent
"""print("Starting Case 11...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == (padNum // 2) - 1):
            padFill.append(0) # puts waterlog pad in the middle
            lightFill.append(0)
        elif(i == padNum // 2):
            lightFill.append(1) # places light frog next to center waterlog pad
            padFill.append(1)
            realNumFrogs = realNumFrogs + 1
        else:
            padFill.append(1)
            lightFill.append(0)
            realNumFrogs = realNumFrogs + 1
        leadFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)
    
print("\n")"""

### CASE 12: waterlogged pad in middle, lead frog adjacent
"""print("Starting Case 12...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == (padNum // 2) - 1):
            padFill.append(0) # puts waterlog pad in the middle
            leadFill.append(0)
        elif(i == padNum // 2):
            leadFill.append(1) # places light frog next to center waterlog pad
            padFill.append(1)
            realNumFrogs = realNumFrogs + 1
        else:
            padFill.append(1)
            leadFill.append(0)
            realNumFrogs = realNumFrogs + 1
        lightFill.append(0)
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)
    
print("\n")"""

### CASE 13: light frog and lead frog adjacent middle
"""print("Starting Case 13...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == (padNum // 2) - 1):
            # puts light frog in the middle
            lightFill.append(1)
            leadFill.append(0)
        elif(i == padNum // 2):
            # places lead frog next to center light frog
            lightFill.append(0)
            leadFill.append(1)
        else:
            lightFill.append(0)
            leadFill.append(0)
            
        padFill.append(1)
        # no waterlogged pads
        realNumFrogs = realNumFrogs + 1
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)
    
print("\n")"""

### CASE 14: light frog and lead frog on either outer ends
"""print("Starting Case 14...")
for padNum in range(3, 11):
    # creation of the lists
    realNumFrogs = 0
    
    padFill = []
    lightFill = []
    leadFill = []
    for i in range(0, padNum):
        if(i == 0):
            # puts light frog at the front
            lightFill.append(1)
            leadFill.append(0)
        elif(i == padNum-1):
            # puts lead frog at the end
            lightFill.append(0)
            leadFill.append(1)
        else:
            lightFill.append(0)
            leadFill.append(0)
            
        padFill.append(1)
        # no waterlogged pads
        realNumFrogs = realNumFrogs + 1
    
    # now we're ready to generate a tree for this layout of frogs
    t_0 = time.perf_counter() # gets time before tree generation
    
    solList = makeAllMoves(padNum, padFill, lightFill, leadFill, realNumFrogs)
    
    t_f = time.perf_counter() # gets time after tree generation
    
    genTime = t_f - t_0
    
    numSols = 0
    for s in solList:
        numSols = numSols + 1
    
    print(genTime, " ", numSols)
    
print("\n")"""

# put this here because the above stuff being in triple """ """ is causing it to output stuff in jupyter notebook
end = 1
       


Enter number of pads: 3
Waterlog pad[ 0 ]? No = 0, Yes = 1: 
0/1?: 0
light Frog on pad[ 0 ]? No = 0, Yes = 1
0/1?: 0
lead Frog on pad[ 0 ]? No = 0, Yes = 1
0/1?: 0
Waterlog pad[ 1 ]? No = 0, Yes = 1: 
0/1?: 0
light Frog on pad[ 1 ]? No = 0, Yes = 1
0/1?: 0
lead Frog on pad[ 1 ]? No = 0, Yes = 1
0/1?: 0
Waterlog pad[ 2 ]? No = 0, Yes = 1: 
0/1?: 0
light Frog on pad[ 2 ]? No = 0, Yes = 1
0/1?: 0
lead Frog on pad[ 2 ]? No = 0, Yes = 1
0/1?: 0


[0, 3, 0]
[0, 2, 1]
[1, 1, 1]
EOP
[0, 0, 3]
[2, 0, 1]
[1, 1, 1]
EOP
[3, 0, 0]
[1, 0, 2]
[1, 1, 1]
EOP
[0, 3, 0]
[1, 2, 0]
[1, 1, 1]
EOP
